In [ ]:
import torch
from image_preprocessing import ImagePreprocessing
from train_functions import color_cluster
from config import __RANDOM_STATE__, root, input_path, output_path
from fine_tune import fine_tune
from inference import inference
from gradcam import grad_cam

In [ ]:
# set seed for reproducibility
torch.manual_seed(__RANDOM_STATE__)
torch.cuda.manual_seed(__RANDOM_STATE__)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [ ]:
# preprocess images
processed_images = ImagePreprocessing(input_path, output_path)
processed_images.cluster_images()  # cluster images
processed_images.apply_filters()  # apply CMY filters to each cluster
color, cluster = color_cluster(root)  # to name the checkpoints folder

In [ ]:
# connect to GPU, else use CPU
device_count = torch.cuda.device_count()  # connect to GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
for i in range(device_count):
    print(f"CUDA device {i}: {torch.cuda.get_device_name(i)}")

In [ ]:
# fine-tune model on dataset specified in `root` variable in `config.py` file
fine_tune(device=device, color=color, cluster=cluster, num_classes=3)

In [ ]:
# inference: get evaluation metrics on test set
inference(N_CLASSES=3, device=device)

In [ ]:
# run Grad-CAM on a sample image
grad_cam(N_CLASSES=3, DEVICE=device)